Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import tensorflow.contrib.model_pruning as pruning
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
# Download pickle from: https://drive.google.com/file/d/1rDhbg-8J4ucd32WTY5Ff_oJMAY_-W0k7/view?usp=sharing

# Change to correct picke location.
pickle_file = '../../../CS4180_Deep_Learning/2_DeepLearningPreliminaries/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    
    del save  # hint to help gc free up memory
    
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)  # None does the same as np.newaxis
    
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by two fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [12]:
# To allow using the default graph again.
tf.reset_default_graph()

batch_size = 10 
patch_size = 5  
depth = 16      
num_hidden = 32
strideConv = [1, 2, 2, 1]

only_visualize_model = False
    
with tf.name_scope('training_data'):
    tf_train_dataset = tf.placeholder(tf.float32, shape=
        (batch_size, image_size, image_size, num_channels),name='train_data')
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels),name='train_labels')

# If we visualize the model, we hide the 
# validation and testing data to simplify the graph.
if (only_visualize_model == False):
    
    # Input data.
    with tf.name_scope('validation_data'):
        tf_valid_dataset = tf.constant(valid_dataset)
    with tf.name_scope('test_data'):
        tf_test_dataset = tf.constant(test_dataset)

# Variables.
layer1_weights = tf.Variable(tf.truncated_normal(
    [patch_size, patch_size, num_channels, depth], stddev=0.1),name='W1')
layer1_biases = tf.Variable(tf.zeros([depth]),name='B1')

layer2_weights = tf.Variable(tf.truncated_normal(
    [patch_size, patch_size, depth, depth], stddev=0.1),name='W2')
layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]),name='B2')

layer3_weights = tf.Variable(tf.truncated_normal(
    [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1),name='W3') # // is integer division!
layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]),name='B3')

layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1),name='W4')
layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]),name='B4')

# Model.
def model(data):
    
    # Layer 1.
    with tf.variable_scope('layer_1') as scope:
        conv1 = tf.nn.conv2d(data, pruning.apply_mask(layer1_weights, scope), strideConv, padding='SAME', name='conv1')
        hidden1 = tf.nn.relu(conv1 + layer1_biases,name=scope.name)
    
    # Layer 2.
    with tf.variable_scope('layer_2') as scope:
        conv2 = tf.nn.conv2d(hidden1, pruning.apply_mask(layer2_weights, scope), strideConv, padding='SAME',name='conv2')
        hidden2 = tf.nn.relu(conv2 + layer2_biases,name=scope.name)
    
    # Layer 3.
    with tf.variable_scope('layer_3'):
        shape = hidden2.get_shape().as_list()
        b2 = shape[0]
        num = shape[1]*shape[2]*shape[3]
        reshape = tf.reshape(hidden2, [b2, num],name='reshape')
        hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases,name='hidden3')
    
    # Layer 4.
    with tf.variable_scope('layer_4'):
        output = tf.add(tf.matmul(hidden3, layer4_weights),layer4_biases,name='output')
    
    return output

# Training computation.
logits = model(tf_train_dataset)

with tf.name_scope('compute_loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits),name='loss')

# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(logits,name='posteriors')

# If we visualize the model, we hide the optimizer, 
# validation and testing data to simplify the graph
if (only_visualize_model == False):
    
    # Optimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate=0.002).minimize(loss)    
    
    with tf.variable_scope('predictions') as scope:
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
        scope.reuse_variables() # to allow reusing model()
        test_prediction = tf.nn.softmax(model(tf_test_dataset))


Now we have built the model, we either visualize it (if only_visualize_model is True), or we train it (if only_visualize_model is False).

In [13]:
# When getting error about the global step: first run
# previous cell to reset the default graph!

num_steps = 2001
pruning_hparams = ''
    
global_step = tf.train.create_global_step()

p = pruning.Pruning(pruning_hparams, global_step=global_step)

mask_update_op = p.conditional_mask_update_op()
p.add_pruning_summaries()

with tf.train.MonitoredTrainingSession() as mon_sess:
    
    # Merge all summaries.
#     summary_op = tf.summary.merge_all()
    
    # Writer.
#     writer = tf.summary.FileWriter('graphs', mon_sess.graph)
    
    # Mask/threshold for printing.
    mask = mon_sess.graph.get_tensor_by_name('layer_1/mask:0')
    threshold = mon_sess.graph.get_tensor_by_name('layer_1/threshold:0')
    
    for step in range(num_steps):
        
        # Train ops.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = mon_sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        # Prune op.
        mon_sess.run(mask_update_op)        

        if step % 50 == 0:
            
            # Summary op.
#             summary_str = mon_sess.run(summary_op)
#             writer.add_summary(summary_str, global_step=step)
            
            print("\nMinibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(session=mon_sess), valid_labels))

            # Print mask/threshold.
            mask_v, threshold_v = mon_sess.run([mask, threshold])
            print("Mask:", np.mean(mask_v))
            print("Threshold:", threshold_v)
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(session=mon_sess), test_labels))
                
#     writer.close()

INFO:tensorflow:Updating masks.
INFO:tensorflow:Summary name layer_1/mask:0/sparsity is illegal; using layer_1/mask_0/sparsity instead.
INFO:tensorflow:Summary name layer_2/mask:0/sparsity is illegal; using layer_2/mask_0/sparsity instead.
INFO:tensorflow:Summary name predictions/layer_1/mask:0/sparsity is illegal; using predictions/layer_1/mask_0/sparsity instead.
INFO:tensorflow:Summary name predictions/layer_2/mask:0/sparsity is illegal; using predictions/layer_2/mask_0/sparsity instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Minibatch loss at step 0: 2.890193
Minibatch accuracy: 10.0%
Validation accuracy: 10.0%
Mask: 1.0
Threshold: 0.0

Minibatch loss at step 50: 2.151886
Minibatch accuracy: 20.0%
Validation accuracy: 40.1%
Mask: 1.0
Threshold: 0.0

Minibatch loss at step 100: 1.190971
Minibatch accuracy: 50.0%
Validation accuracy: 64.2%
Mask: 1.0
Threshold: 0.0

Minibatch loss at step 150: 1.141832
M